In [1]:
# stage 1 training and prediction
import pandas as pd
import numpy as np
import xgboost as xgb
import shap
import ta
import joblib
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
import random
import os
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.model_selection import ParameterGrid, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import numpy as np

REG_PARAM_GRID = {
    "max_depth": [3, 4, 5],
    "learning_rate": [0.05, 0.1],
    "n_estimators": [600, 1000],
    "subsample": [0.8, 0.9],
    "colsample_bytree": [0.8, 1.0],
    "min_child_weight": [1.0],
    "reg_lambda": [1.0, 2.0],
}
SEEDS = [42, 202]

GLOBAL_SEED = 42
np.random.seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)
os.environ['PYTHONHASHSEED'] = str(GLOBAL_SEED)
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
# ------------------------------------------------------------------------
# 0. Load and align data
# ------------------------------------------------------------------------
factors = pd.read_csv("aligned_factors.csv", index_col=0, parse_dates=True)
returns = pd.read_csv("daily_returns_10ETFs.csv", index_col=0, parse_dates=True)

# Align dates to ensure matching indices
dates = factors.index.intersection(returns.index)
factors = factors.loc[dates]
returns = returns.loc[dates]


# ------------------------------------------------------------------------
# 1. Compute technical indicators and lagged features per ETF
# ------------------------------------------------------------------------
all_tech_features = []

for etf in returns.columns:
    close = (1 + returns[etf]).cumprod()
    tech_df = pd.DataFrame(index=returns.index)

    # Selected indicators (others commented out to reduce noise)
    tech_df[f'{etf}_SMA_5']   = ta.trend.sma_indicator(close, window=5)
    tech_df[f'{etf}_EMA_12']  = ta.trend.ema_indicator(close, window=12)
    tech_df[f'{etf}_RSI_7']   = ta.momentum.rsi(close, window=7)
    tech_df[f'{etf}_MACD']    = ta.trend.macd_diff(close)
    tech_df[f'{etf}_ATR']     = ta.volatility.average_true_range(
        high=close * 1.01, low=close * 0.99, close=close, window=10
    )
    tech_df[f'{etf}_Vol_5']   = returns[etf].rolling(window=5).std()
    tech_df[f'{etf}_Mom_3']   = returns[etf].rolling(window=3).mean()

    # Lagged returns (shifted so only past information is used)
    for lag in [1, 2, 3]:
        tech_df[f'{etf}_LagRet_{lag}'] = returns[etf].shift(lag)

    all_tech_features.append(tech_df)

# Concatenate technical indicators for all ETFs
technical_features = pd.concat(all_tech_features, axis=1)

# ------------------------------------------------------------------------
# 2. Create lagged factor features
# ------------------------------------------------------------------------
for factor in ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']:
    for lag in [1, 2, 3]:
        factors[f'{factor}_lag_{lag}'] = factors[factor].shift(lag)

# Drop rows with NA values arising from lagging
factors = factors.dropna()

# ------------------------------------------------------------------------
# 3. Combine factors, technical features, and VIX change
# ------------------------------------------------------------------------
features = pd.concat([factors, technical_features], axis=1).dropna()
vix = pd.read_csv("VIX_History.csv", index_col=0, parse_dates=True)

# Align VIX to our feature dates and compute lagged change
vix_aligned = vix['CLOSE'].reindex(features.index).ffill()
features['VIX'] = vix_aligned.pct_change(fill_method=None).shift(1)
features['VIX'] = features['VIX'].fillna(0)

# Define the target: next-day return per ETF
target_returns = returns.shift(-1).loc[features.index].dropna()
features = features.loc[target_returns.index]

train_years = 12      # years used for training
valid_years = 1       # years used for validation
test_years  = 1       # years used for testing/prediction
retrain_frequency = 1 # years between retrainings
start_year = 2009
end_year   = 2024

# ========= SHAP gate: pick top-N features across ETF-specific + Fama-French =========
import numpy as np
import pandas as pd
import xgboost as xgb
import shap

# How many total features to keep (ETF + global, combined)
# TOP_N_FEATURES = 10   # <- tune this (e.g., 12–20 works well)

# ---- Globals & helpers needed by the SHAP gate ----

# Global (macro) factors you maintain in your `features` frame
GLOBAL_FACTORS = [
    "Mkt-RF", "SMB", "HML", "RMW", "CMA",
    "Mkt-RF_lag_1", "Mkt-RF_lag_2", "Mkt-RF_lag_3",
    "SMB_lag_1", "SMB_lag_2", "SMB_lag_3",
    "HML_lag_1", "HML_lag_2", "HML_lag_3",
    "RMW_lag_1", "RMW_lag_2", "RMW_lag_3",
    "CMA_lag_1", "CMA_lag_2", "CMA_lag_3",
    "VIX"  # include if present; harmless if missing later
]

# ETF-specific feature suffixes (no ETF prefix here; we’ll prepend per-ETF when building panel)
ETF_SUFFIX_POOL = [
    "SMA_5","EMA_12","RSI_7","MACD","ATR","Vol_5","Mom_3","LagRet_1","LagRet_2","LagRet_3"
]

def _robust_cs(x):
    med = x.median()
    mad = (x - med).abs().median()
    return (x - med) / (mad + 1e-6)

def _make_forward_returns(daily_ret_df: pd.DataFrame, horizon=21, method="compound"):
    if method == "compound":
        cum = (1.0 + daily_ret_df).cumprod()
        return cum.shift(-horizon) / cum - 1.0
    return daily_ret_df.rolling(horizon).sum().shift(-horizon)

def _build_panel_for_gate(features_df, returns_df, etfs, start, end, forward_h=21, use_risk_adj=True):
    """Same idea as your build_panel, used only for the SHAP gate on a base window."""
    feat = features_df.loc[start:end].copy()
    rets = returns_df.loc[start:end].copy()

    # forward returns (compound)
    cum = (1.0 + rets).cumprod()
    fwd = cum.shift(-forward_h) / cum - 1.0
    vol20 = rets.rolling(20).std()

    rows = []
    for dt in feat.index:
        if dt not in fwd.index: 
            continue
        for etf in etfs:
            row = {"Date": dt, "ETF": etf}
            # ETF-specific columns
            for suf in ETF_SUFFIX_POOL:
                col = f"{etf}_{suf}"
                if col not in feat.columns:
                    continue
                row[suf] = feat.at[dt, col]
            # global factors (no ETF prefix) if present in features
            for gf in GLOBAL_FACTORS:
                if gf in feat.columns:
                    row[gf] = feat.at[dt, gf]
            # label
            y = fwd.at[dt, etf]
            if use_risk_adj:
                v = vol20.at[dt, etf]
                y = np.nan if (pd.isna(y) or pd.isna(v)) else (y / max(v, 1e-6))
            row["y"] = y
            rows.append(row)

    panel = pd.DataFrame(rows).dropna()
    if panel.empty:
        raise ValueError("SHAP gate panel is empty—check date ranges and column names.")

    feat_cols = [c for c in panel.columns if c not in ["Date","ETF","y"]]
    # do NOT cross-sec standardize global factors; only ETF‑specific features
    cs_cols = [c for c in feat_cols if c not in GLOBAL_FACTORS]
    panel[cs_cols] = panel.groupby("Date")[cs_cols].transform(_robust_cs)

    # Light winsorization on ETF‑specific features
    for c in cs_cols:
        q1, q99 = panel[c].quantile([0.01, 0.99])
        panel[c] = panel[c].clip(q1, q99)

    panel.sort_values(["Date","ETF"], inplace=True)
    panel.reset_index(drop=True, inplace=True)
    return panel

def run_shap_gate_select_topN(features_df, returns_df, etfs,
                              start_year, train_years, valid_years,
                              forward_h=21, use_risk_adj=True,
                              top_n=10, min_suffixes=6, max_globals=4, seed=42):
    """
    Enforce a balanced mix: at least `min_suffixes` ETF-specific suffixes,
    and at most `max_globals` macro families. Families are MKT/SMB/HML/RMW/CMA(+VIX if present).
    We return:
      - selected ETF suffixes (names like 'RSI_7', 'MACD', ...)
      - selected macro columns (best single lag per family, e.g., 'Mkt-RF_lag_2')
      - a table with the gate-ranking for debugging
    """
    assert min_suffixes + max_globals <= top_n, "Choose top_n >= min_suffixes + max_globals"

    base_train_start = pd.Timestamp(start_year - train_years, 1, 1)
    base_train_end   = pd.Timestamp(start_year - valid_years - 1, 12, 31)

    panel = _build_panel_for_gate(features_df, returns_df, etfs,
                                  base_train_start, base_train_end,
                                  forward_h=forward_h, use_risk_adj=use_risk_adj)
    feat_cols = [c for c in panel.columns if c not in ["Date","ETF","y"]]
    Xb, yb = panel[feat_cols].values, panel["y"].values

    # quick regression for SHAP importances
    params = dict(
        objective="reg:squarederror",
        tree_method="hist",
        max_depth=4,
        eta=0.08,
        subsample=0.9,
        colsample_bytree=0.9,
        seed=int(seed),
        sampling_method="uniform",
    )
    dtrain = xgb.DMatrix(Xb, label=yb)
    model = xgb.train(params, dtrain, num_boost_round=400, verbose_eval=False)

    # SHAP importances per column
    explainer = shap.TreeExplainer(model)
    sv = explainer.shap_values(Xb)               # (n, d)
    shap_mean_abs = np.abs(sv).mean(axis=0)      # (d,)
    col_imp = pd.Series(shap_mean_abs, index=feat_cols).sort_values(ascending=False)

    # --- ETF-specific suffix importance (one column per suffix) ---
    suf_imp = {s: float(col_imp.get(s, 0.0)) for s in ETF_SUFFIX_POOL if s in col_imp.index}
    suf_rank = pd.Series(suf_imp).sort_values(ascending=False)

    # --- Macro family importance (aggregate all lags within a family) ---
    fam_to_cols = {
        "MKT": [c for c in feat_cols if c.startswith("Mkt-RF")],
        "SMB": [c for c in feat_cols if c.startswith("SMB")],
        "HML": [c for c in feat_cols if c.startswith("HML")],
        "RMW": [c for c in feat_cols if c.startswith("RMW")],
        "CMA": [c for c in feat_cols if c.startswith("CMA")],
    }
    if "VIX" in feat_cols:
        fam_to_cols["VIX"] = ["VIX"]

    fam_imp = {}
    fam_best_col = {}
    for fam, cols in fam_to_cols.items():
        cols = [c for c in cols if c in col_imp.index]
        if not cols:
            continue
        fam_imp[fam] = float(col_imp[cols].sum())
        fam_best_col[fam] = col_imp[cols].idxmax()   # best single lag/column within the family

    fam_rank = pd.Series(fam_imp).sort_values(ascending=False)

    # --- Enforce quotas ---
    sel_suffixes = list(suf_rank.head(min_suffixes).index)
    sel_fams     = list(fam_rank.head(max_globals).index)

    # If suffixes are scarce, backfill from pool order
    if len(sel_suffixes) < min_suffixes:
        backfill = [s for s in ETF_SUFFIX_POOL if s not in sel_suffixes]
        sel_suffixes += backfill[:(min_suffixes - len(sel_suffixes))]
    sel_suffixes = sel_suffixes[:min_suffixes]

    # Map families to a single best column per family
    sel_global_cols = [fam_best_col[f] for f in sel_fams if f in fam_best_col]

    # Pretty table for logging
    gate_table = pd.concat([
        suf_rank.rename("mean_abs_shap").to_frame().assign(kind="suffix"),
        fam_rank.rename("mean_abs_shap").to_frame().assign(kind="family"),
    ], axis=0)

    return sel_suffixes, sel_global_cols, gate_table



# Infer ETF universe (if you don’t already have it)
etf_list = list(returns.columns)




In [2]:
# ===================== Unified SHAP Gate -> Per-ETF Stage-1 =====================
import numpy as np
import pandas as pd
import xgboost as xgb
import shap
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import ParameterGrid
from scipy.stats import spearmanr
import time, os, json
import os, numpy as np, pandas as pd, xgboost as xgb, shap
# ---------------- Config ----------------
FWD_H_ETF          = 1      # predict next-day return; switch to 21 for 1-month forward
USE_RISK_ADJ_ETF   = False  # risk-adjusted label? (start False for clarity)
TOP_N_GENERIC      = 10     # <- your N
MIN_SUFFIXES       = 6      # ensure enough ETF-specific cross-sectional info
MAX_GLOBAL_FAMILIES= 4      # cap macro families kept
EMA_SMOOTH_SPAN    = 0      # optional smoothing of Predicted_Score; 0 disables
ART_DIR            = "stage1_models_unified_gate"
os.makedirs(ART_DIR, exist_ok=True)

XGB_TREE_METHOD = "hist"    # "hist" (CPU) is deterministic; GPU can introduce tiny nondeterminism

REG_PARAM_GRID = {
    "max_depth": [3, 4, 5],
    "learning_rate": [0.05, 0.1],
    "n_estimators": [600, 1000],
    "subsample": [0.8, 0.9],
    "colsample_bytree": [0.8, 1.0],
    "min_child_weight": [1.0],
    "reg_lambda": [1.0, 2.0],
}
SEEDS = [42, 202]




GLOBAL_SEED = 42
np.random.seed(GLOBAL_SEED)

# Fixed pools (keep as you already defined)
ETF_SUFFIX_POOL = [
    "SMA_5","EMA_12","RSI_7","MACD","ATR","Vol_5","Mom_3","LagRet_1","LagRet_2","LagRet_3"
]
MACRO_FAMILY_MAP = {
    "MKT": ["Mkt-RF", "Mkt-RF_lag_1","Mkt-RF_lag_2","Mkt-RF_lag_3"],
    "SMB": ["SMB",    "SMB_lag_1","SMB_lag_2","SMB_lag_3"],
    "HML": ["HML",    "HML_lag_1","HML_lag_2","HML_lag_3"],
    "RMW": ["RMW",    "RMW_lag_1","RMW_lag_2","RMW_lag_3"],
    "CMA": ["CMA",    "CMA_lag_1","CMA_lag_2","CMA_lag_3"],
}
if "VIX" in features.columns:
    MACRO_FAMILY_MAP["VIX"] = ["VIX"]

TOP_N_GENERIC       = 10   # your N
MIN_SUFFIXES        = 6
MAX_GLOBAL_FAMILIES = 4
EPS_TIE = 1e-12            # treat values within this as equal

# Stable, deterministic XGB for the gate (no subsampling, single thread)
GATE_XGB = dict(
    objective="reg:squarederror",
    tree_method="hist",
    device="cuda",
    n_estimators=512,
    max_depth=4,
    learning_rate=0.08,
    subsample=1.0,
    colsample_bytree=1.0,
    min_child_weight=1.0,
    reg_lambda=1.0,
    random_state=GLOBAL_SEED,
    seed=GLOBAL_SEED,
    n_jobs=1,
)

# def _forward_returns(series: pd.Series, horizon=1, risk_adj=False):
#     if horizon == 1:
#         fwd = series.shift(-1)
#     else:
#         cum = (1.0 + series).cumprod()
#         fwd = cum.shift(-horizon) / cum - 1.0
#     if risk_adj:
#         vol20 = series.rolling(20).std()
#         fwd = fwd / (vol20.replace(0, np.nan) + 1e-6)
#     return fwd

def _stable_sort_pairs(pairs):
    """pairs: list of (name, score). Sort by score desc, then name asc."""
    return sorted(pairs, key=lambda t: (-t[1], t[0]))


# ---------------- Helpers ----------------
def _forward_returns_1d(series: pd.Series, horizon=1, risk_adj=False, vol20: pd.Series=None):
    if horizon == 1:
        fwd = series.shift(-1)
    else:
        cum = (1.0 + series).cumprod()
        fwd = cum.shift(-horizon) / cum - 1.0
    if risk_adj:
        if vol20 is None:
            vol20 = series.rolling(20).std()
        fwd = fwd / (vol20.replace(0, np.nan) + 1e-6)
    return fwd

def _ts_ic(y, p):
    y, p = np.asarray(y), np.asarray(p)
    m = np.isfinite(y) & np.isfinite(p)
    y, p = y[m], p[m]
    if len(y) < 5 or np.std(y) < 1e-12 or np.std(p) < 1e-12:
        return np.nan
    ic, _ = spearmanr(y, p)
    return float(ic) if np.isfinite(ic) else np.nan

def build_ds_from_cols(etf: str, start, end, selected_suffixes: list, selected_macro_cols: list,
                       horizon=1, risk_adj=False, imputer: SimpleImputer=None, fit_imputer=False):
    """
    Dataset with FIXED column order: suffix names + selected macro column names.
    Uses train-median imputation without indicators to keep shape stable.
    """
    feat = features.loc[start:end]
    y    = _forward_returns_1d(
              returns[etf], horizon=horizon, risk_adj=risk_adj,
              vol20=returns[etf].rolling(20).std()
           ).loc[start:end]

    cols = selected_suffixes + selected_macro_cols
    X = pd.DataFrame(index=feat.index, columns=cols, dtype=float)

    # Fill from source features
    for s in selected_suffixes:
        c = f"{etf}_{s}"
        if c in feat.columns:
            X[s] = feat[c].values
    for g in selected_macro_cols:
        if g in feat.columns:
            X[g] = feat[g].values

    df = pd.concat([X, y.rename("y")], axis=1)

    # Imputer: median (no indicators). Pre-fill all-NaN train columns to avoid fit errors.
    if imputer is None:
        imputer = SimpleImputer(strategy="median")  # <-- NO add_indicator

    if fit_imputer:
        X_train = df[cols].copy()
        all_nan_cols = [c for c in cols if X_train[c].isna().all()]
        for c in all_nan_cols:
            X_train[c] = 0.0  # safe fallback seed for median
        X_imp = pd.DataFrame(imputer.fit_transform(X_train), index=df.index, columns=cols)
    else:
        # transform expects same columns in same order as fit
        X_imp = pd.DataFrame(imputer.transform(df[cols]), index=df.index, columns=cols)

    out = pd.concat([X_imp, df["y"]], axis=1).dropna(subset=["y"])
    return out, imputer

def daily_cs_rank(x):
    return x.rank(pct=True) - 0.5

def aggregate_shap_by_cols(shap_df: pd.DataFrame,
                           selected_suffixes: list,
                           selected_macro_cols: list) -> pd.DataFrame:
    """
    Emit SHAP_<suffix> for each selected ETF suffix feature,
    and SHAP_<macrocol> for each selected macro *column* (e.g., SHAP_SMB_lag_2).
    No family-level aggregation is performed here.
    """
    out = pd.DataFrame(index=shap_df.index)

    # ETF suffix columns (these are named by suffix in shap_df)
    for s in selected_suffixes:
        out[f"SHAP_{s}"] = shap_df[s] if s in shap_df.columns else 0.0

    # Exact macro columns selected by the gate (e.g., "SMB_lag_2")
    for c in selected_macro_cols:
        out[f"SHAP_{c}"] = shap_df[c] if c in shap_df.columns else 0.0

    # Optional diagnostic total (sum over raw feature columns in shap_df)
    out["SHAP_Total"] = shap_df.sum(axis=1) if shap_df.shape[1] else 0.0
    return out.fillna(0.0)


def build_stable_unified_gate(start_year, train_years, valid_years,
                              horizon=1, risk_adj=False,
                              top_n=10, min_suffixes=6, max_global_families=4):
    # cache: if files exist, reuse
    if os.path.exists("unified_gate_suffixes.csv") and os.path.exists("unified_gate_macro_cols.csv"):
        suffixes = pd.read_csv("unified_gate_suffixes.csv")["SELECTED_SUFFIXES"].tolist()
        macro_cols = pd.read_csv("unified_gate_macro_cols.csv")["SELECTED_MACRO_COLS"].tolist()
        macro_fams = []
        for fam, cols in MACRO_FAMILY_MAP.items():
            if any(c in macro_cols for c in cols):
                macro_fams.append(fam)
        return suffixes, macro_cols, macro_fams

    base_train_start = pd.Timestamp(start_year - train_years, 1, 1)
    base_train_end   = pd.Timestamp(start_year - valid_years - 1, 12, 31)

    etfs = sorted(list(returns.columns))  # stable order
    # group accumulators
    group_sum = {s:0.0 for s in ETF_SUFFIX_POOL}
    group_cnt = {s:0   for s in ETF_SUFFIX_POOL}
    for fam in MACRO_FAMILY_MAP.keys():
        group_sum[fam] = 0.0
        group_cnt[fam] = 0
    macro_col_sum = {c:0.0 for fam in MACRO_FAMILY_MAP.values() for c in fam}
    macro_col_cnt = {c:0   for fam in MACRO_FAMILY_MAP.values() for c in fam}

    for etf in etfs:
        # fixed column order: suffixes (alphabetical by our list), then all macro cols in family order
        etf_cols = []
        for s in ETF_SUFFIX_POOL:
            c = f"{etf}_{s}"
            if c in features.columns:
                etf_cols.append(c)
        for fam in MACRO_FAMILY_MAP.keys():
            for c in MACRO_FAMILY_MAP[fam]:
                if c in features.columns:
                    etf_cols.append(c)

        Xb = features.loc[base_train_start:base_train_end, etf_cols].copy()
        yb = _forward_returns_1d(returns[etf], horizon=horizon, risk_adj=risk_adj).loc[base_train_start:base_train_end]
        dfb = pd.concat([Xb, yb.rename("y")], axis=1).dropna()
        if dfb.empty:
            continue

        # enforce fixed column order
        cols = [c for c in etf_cols if c in dfb.columns and c != "y"]
        Xb = dfb[cols].values
        yb = dfb["y"].values

        model = xgb.XGBRegressor(**GATE_XGB)
        model.fit(Xb, yb)
        exp = shap.TreeExplainer(model)
        sv = exp.shap_values(Xb)
        imp = np.abs(sv).mean(axis=0)
        col_imp = list(zip(cols, imp))

        # suffix groups (each suffix has exactly one column per ETF)
        for s in ETF_SUFFIX_POOL:
            c = f"{etf}_{s}"
            if c in cols:
                v = imp[cols.index(c)]
                group_sum[s] += float(v); group_cnt[s] += 1

        # macro families: sum their columns; also track each column
        for fam, fam_cols in MACRO_FAMILY_MAP.items():
            vals = []
            for c in fam_cols:
                if c in cols:
                    v = imp[cols.index(c)]
                    vals.append(v)
                    macro_col_sum[c] += float(v)
                    macro_col_cnt[c] += 1
            if vals:
                group_sum[fam] += float(np.sum(vals)); group_cnt[fam] += 1

    # average across ETFs
    group_avg = {g: (group_sum[g] / max(group_cnt[g],1)) for g in group_sum.keys()}

    # stable rankings with tie‑break
    suffix_pairs = [(s, group_avg[s]) for s in ETF_SUFFIX_POOL]
    macro_pairs  = [(fam, group_avg[fam]) for fam in MACRO_FAMILY_MAP.keys()]

    suffix_rank = _stable_sort_pairs(suffix_pairs)
    macro_rank  = _stable_sort_pairs(macro_pairs)

    chosen_suffixes = [name for name,_ in suffix_rank[:min_suffixes]]
    chosen_fams     = [name for name,_ in macro_rank[:max_global_families]]

    sel_groups = chosen_suffixes + chosen_fams
    for name,_ in (suffix_rank + macro_rank):
        if len(sel_groups) >= top_n: break
        if name not in sel_groups:
            sel_groups.append(name)
    sel_groups = sel_groups[:top_n]

    # pick the ONE best column per selected macro family (stable tie break)
    selected_macro_cols = []
    for fam in sel_groups:
        if fam in MACRO_FAMILY_MAP:
            items = []
            for c in MACRO_FAMILY_MAP[fam]:
                v = macro_col_sum.get(c, 0.0) / max(macro_col_cnt.get(c, 1), 1)
                items.append((c, v))
            # stable pick: sort by score desc, then column name asc
            c_best = _stable_sort_pairs(items)[0][0]
            selected_macro_cols.append(c_best)

    selected_suffixes = [g for g in sel_groups if g in ETF_SUFFIX_POOL]
    selected_macro_fams = [g for g in sel_groups if g in MACRO_FAMILY_MAP]

    # cache to disk
    pd.Series(selected_suffixes, name="SELECTED_SUFFIXES").to_csv("unified_gate_suffixes.csv", index=False)
    pd.Series(selected_macro_cols, name="SELECTED_MACRO_COLS").to_csv("unified_gate_macro_cols.csv", index=False)

    return selected_suffixes, selected_macro_cols, selected_macro_fams

# ---- call it (use your window settings) ----
SELECTED_SUFFIXES, SELECTED_MACRO_COLS, SELECTED_MACRO_FAMILIES = build_stable_unified_gate(
    start_year=start_year,
    train_years=train_years,
    valid_years=valid_years,
    horizon=FWD_H_ETF,
    risk_adj=USE_RISK_ADJ_ETF,
    top_n=TOP_N_GENERIC,
    min_suffixes=MIN_SUFFIXES,
    max_global_families=MAX_GLOBAL_FAMILIES,
)
print("Unified gate selected suffixes:", SELECTED_SUFFIXES)
print("Unified gate selected macro columns:", SELECTED_MACRO_COLS)


# ---------------- 2) Rolling per-ETF training with fixed selected features ----------------
def mean_daily_ic(dates: np.ndarray, y_true: np.ndarray, y_pred: np.ndarray) -> float:
    df = pd.DataFrame({"Date": dates, "y": y_true, "p": y_pred})
    vals = []
    for dt, g in df.groupby("Date"):
        y, p = g["y"].values, g["p"].values
        m = np.isfinite(y) & np.isfinite(p)
        y, p = y[m], p[m]
        if len(y) >= 3 and np.std(y) > 1e-12 and np.std(p) > 1e-12:
            ic, _ = spearmanr(y, p)
            if np.isfinite(ic): vals.append(ic)
    return float(np.mean(vals)) if vals else np.nan

all_rows, metrics_rows = [], []
year = start_year
while year <= end_year - test_years + 1:
    print(f"\n=== Stage‑1 (Unified‑gate) per‑ETF regression | window {year} ===")
    t0 = time.time()

    train_start = pd.Timestamp(year - train_years, 1, 1)
    train_end   = pd.Timestamp(year - valid_years - 1, 12, 31)
    valid_start = pd.Timestamp(year - valid_years, 1, 1)
    valid_end   = pd.Timestamp(year - 1, 12, 31)
    test_start  = pd.Timestamp(year, 1, 1)
    test_end    = pd.Timestamp(year + test_years - 1, 12, 31)

    for etf in etf_list:
        # Build datasets with fixed selected features and train-median imputation
        ds_tr, imp = build_ds_from_cols(etf, train_start, train_end, SELECTED_SUFFIXES, SELECTED_MACRO_COLS,
                                        horizon=FWD_H_ETF, risk_adj=USE_RISK_ADJ_ETF,
                                        imputer=None, fit_imputer=True)
        ds_va, _   = build_ds_from_cols(etf, valid_start, valid_end, SELECTED_SUFFIXES, SELECTED_MACRO_COLS,
                                        horizon=FWD_H_ETF, risk_adj=USE_RISK_ADJ_ETF,
                                        imputer=imp, fit_imputer=False)
        ds_te, _   = build_ds_from_cols(etf, test_start,  test_end,  SELECTED_SUFFIXES, SELECTED_MACRO_COLS,
                                        horizon=FWD_H_ETF, risk_adj=USE_RISK_ADJ_ETF,
                                        imputer=imp, fit_imputer=False)
        
        purge = int(FWD_H_ETF)
        if len(ds_tr) > purge:
            ds_tr = ds_tr.iloc[:-purge]
        if len(ds_va) > purge:
            ds_va = ds_va.iloc[:-purge]
            
        if ds_tr.empty or ds_va.empty or ds_te.empty:
            print(f"[{etf}][{year}] Insufficient data; skipping.")
            continue

        feat_cols = SELECTED_SUFFIXES + SELECTED_MACRO_COLS  # fixed order everywhere

        scaler = StandardScaler()
        X_tr = scaler.fit_transform(ds_tr[feat_cols].values); y_tr = ds_tr["y"].values
        X_va = scaler.transform(ds_va[feat_cols].values);     y_va = ds_va["y"].values
        X_te = scaler.transform(ds_te[feat_cols].values);     y_te = ds_te["y"].values

        # Hyperparam search: maximize validation time-series IC (tie-break RMSE)
        best_ic, best_rmse = -np.inf, np.inf
        best_model, best_params = None, None
        for params in ParameterGrid(REG_PARAM_GRID):
            for seed in SEEDS:
                dtr = xgb.DMatrix(X_tr, label=y_tr)
                dva = xgb.DMatrix(X_va, label=y_va)
                xgb_params = {
                    "objective": "reg:squarederror",
                    "tree_method": XGB_TREE_METHOD,
                    "eval_metric": "rmse",
                    "seed": int(seed),
                    "sampling_method": "uniform",
                    "max_depth": int(params["max_depth"]),
                    "eta": float(params["learning_rate"]),
                    "subsample": float(params["subsample"]),
                    "colsample_bytree": float(params["colsample_bytree"]),
                    "min_child_weight": float(params["min_child_weight"]),
                    "lambda": float(params["reg_lambda"]),
                    "base_score": float(np.nanmean(y_tr)),
                }
                num_boost_round = int(params["n_estimators"])
                model = xgb.train(
                    xgb_params, dtr,
                    num_boost_round=max(200, num_boost_round),
                    evals=[(dtr,"train"), (dva,"valid")],
                    early_stopping_rounds=min(100, num_boost_round//3),
                    verbose_eval=False
                )

                p_va = model.predict(dva, iteration_range=(0, (model.best_iteration or 0) + 1))
                ic_va = _ts_ic(y_va, p_va)
                rmse_va = float(np.sqrt(np.mean((y_va - p_va)**2)))
                if (ic_va > best_ic) or (np.isclose(ic_va, best_ic) and rmse_va < best_rmse):
                    best_ic, best_rmse = ic_va, rmse_va
                    best_model, best_params = model, {"seed": seed, **params}

        # Test predictions
        # dte = xgb.DMatrix(X_te, label=y_te)
        # p_te = best_model.predict(dte, iteration_range=(0, (best_model.best_iteration or 0) + 1))

        # ---------------- Test prediction with valid t+1 only ----------------
        # Next-day realized returns (compute once over the full series)
        next_day_actual_series = returns[etf].shift(-1)
        
        # Restrict the test set to rows that have a valid next-day actual
        actual_on_te = next_day_actual_series.reindex(ds_te.index)
        valid_mask = actual_on_te.notna().values
        if not np.all(valid_mask):
            n_drop = int((~valid_mask).sum())
            if n_drop > 0:
                print(f"[{etf}][{year}] Dropping {n_drop} test rows with missing Actual_Return (no t+1).")
        
        # Filter test arrays BEFORE inference/SHAP so shapes align
        X_te_valid = X_te[valid_mask]
        y_te_valid = y_te[valid_mask]
        dates_valid = ds_te.index[valid_mask]
        actual_valid = actual_on_te.loc[dates_valid].values  # guaranteed non-NaN now
        
        # Predict on valid rows only
        dte = xgb.DMatrix(X_te_valid, label=y_te_valid)
        p_te = best_model.predict(dte, iteration_range=(0, (best_model.best_iteration or 0) + 1))
        
        # SHAP on valid rows only (columns = feat_cols in fixed order)
        explainer = shap.TreeExplainer(best_model)
        shap_vals = explainer.shap_values(X_te_valid)
        shap_df   = pd.DataFrame(shap_vals, columns=feat_cols, index=dates_valid)
        
        # Aggregate SHAP using the exact selected macro columns (not families)
        shap_agg = aggregate_shap_by_cols(shap_df, SELECTED_SUFFIXES, SELECTED_MACRO_COLS)
        exp_val  = explainer.expected_value
        shap_agg["SHAP_Base"] = float(exp_val if np.ndim(exp_val)==0 else exp_val[0])

        
        # Assemble output rows (no NaNs in Actual_Return)
        out = pd.DataFrame({
            "Date": dates_valid,
            "ETF": etf,
            "Year": year,
            "Predicted_Return": p_te,          # raw regression prediction
            "Actual_Return": actual_valid      # clean, with valid t+1 only
        }).reset_index(drop=True)
        
        # Append SHAP aggregates
        out = pd.concat([out, shap_agg.reset_index(drop=True)], axis=1)
        all_rows.append(out)
        
        if not all_rows:
            raise RuntimeError("No test rows produced; check data ranges and feature availability.")

        # Compute test metrics on the valid rows only
        ic_test   = _ts_ic(y_te_valid, p_te)
        rmse_test = float(np.sqrt(np.mean((y_te_valid - p_te)**2)))
        metrics_rows.append({
            "ETF": etf, "Year": year,
            "IC_valid": float(best_ic), "RMSE_valid": float(best_rmse),
            "IC_test": float(ic_test), "RMSE_test": float(rmse_test),
            "train_range": f"{train_start.date()}..{train_end.date()}",
            "valid_range": f"{valid_start.date()}..{valid_end.date()}",
            "test_range":  f"{test_start.date()}..{test_end.date()}",
            "selected_suffixes": ",".join(SELECTED_SUFFIXES),
            "selected_macro_cols": ",".join(SELECTED_MACRO_COLS),
        })


        # # SHAP on test (columns exactly = feat_cols)
        # explainer = shap.TreeExplainer(best_model)
        # shap_vals = explainer.shap_values(X_te)   # (n_test, n_features)
        # shap_df   = pd.DataFrame(shap_vals, columns=feat_cols, index=ds_te.index)
        # # Robust: ensure every expected column exists (fill 0)
        # for c in feat_cols:
        #     if c not in shap_df.columns:
        #         shap_df[c] = 0.0
        # shap_df = shap_df[feat_cols]
        # 
        # shap_agg = aggregate_shap_fixed(shap_df, SELECTED_SUFFIXES, SELECTED_MACRO_COLS)
        # exp_val  = explainer.expected_value
        # shap_agg["SHAP_Base"] = float(exp_val if np.ndim(exp_val)==0 else exp_val[0])
        # 
        # # Assemble output
        # out = pd.DataFrame({
        #     "Date": ds_te.index,
        #     "ETF": etf,
        #     "Year": year,
        #     "Predicted_Return": p_te,  # raw reg prediction
        #     "Actual_Return": returns.loc[ds_te.index, etf].shift(-1).values
        # }).reset_index(drop=True)
        # out = pd.concat([out, shap_agg.reset_index(drop=True)], axis=1)
        # 
        # all_rows.append(out)
        # 
        # # Metrics
        # ic_test   = _ts_ic(y_te, p_te)
        # rmse_test = float(np.sqrt(np.mean((y_te - p_te)**2)))
        # metrics_rows.append({
        #     "ETF": etf, "Year": year,
        #     "IC_valid": float(best_ic), "RMSE_valid": float(best_rmse),
        #     "IC_test": float(ic_test), "RMSE_test": float(rmse_test),
        #     "train_range": f"{train_start.date()}..{train_end.date()}",
        #     "valid_range": f"{valid_start.date()}..{valid_end.date()}",
        #     "test_range":  f"{test_start.date()}..{test_end.date()}",
        #     "selected_suffixes": ",".join(SELECTED_SUFFIXES),
        #     "selected_macro_cols": ",".join(SELECTED_MACRO_COLS),
        # })

        # Save artifacts per ETF/year
        wdir = os.path.join(ART_DIR, etf, f"year_{year}")
        os.makedirs(wdir, exist_ok=True)
        best_model.save_model(os.path.join(wdir, "xgb_regressor.json"))
        with open(os.path.join(wdir, "meta.json"), "w") as f:
            json.dump({
                "best_params": best_params,
                "ic_valid": float(best_ic),
                "rmse_valid": float(best_rmse),
                "feature_cols": feat_cols,
                "objective": "reg:squarederror",
                "selected_suffixes": SELECTED_SUFFIXES,
                "selected_macro_cols": SELECTED_MACRO_COLS
            }, f, indent=2)
            
        import joblib, json, os

        # Save the trained model
        model_dir = os.path.join(ART_DIR, etf, f"year_{year}")
        os.makedirs(model_dir, exist_ok=True)
        best_model.save_model(os.path.join(model_dir, "xgb_regressor.json"))
        
        # Save scaler and imputer used for this ETF/year
        joblib.dump(scaler, os.path.join(model_dir, "scaler.pkl"))
        joblib.dump(imp, os.path.join(model_dir, "imputer.pkl"))
        
        # Save metadata
        meta = {
            "best_params": best_params,
            "ic_valid": float(best_ic),
            "rmse_valid": float(best_rmse),
            "ic_test": float(ic_test),
            "rmse_test": float(rmse_test),
            "train_range": [str(train_start.date()), str(train_end.date())],
            "valid_range": [str(valid_start.date()), str(valid_end.date())],
            "test_range":  [str(test_start.date()),  str(test_end.date())],
            "selected_suffixes": SELECTED_SUFFIXES,
            "selected_macro_cols": SELECTED_MACRO_COLS,
            "feature_cols_order": feat_cols
        }

        with open(os.path.join(model_dir, "meta.json"), "w") as f:
            json.dump(meta, f, indent=2)

    print(f"Window {year} done in {time.time()-t0:.2f}s")
    year += retrain_frequency

# ---------------- Combine & finalize for Stage‑2 ----------------
final = pd.concat(all_rows, ignore_index=True)

# Cross‑sectional daily rank score in [-0.5, 0.5]
final["Predicted_Score"] = final.groupby("Date")["Predicted_Return"].transform(lambda s: s.rank(pct=True) - 0.5)
if EMA_SMOOTH_SPAN and EMA_SMOOTH_SPAN > 1:
    final.sort_values(["ETF","Date"], inplace=True)
    final["Predicted_Score"] = (
        final.groupby("ETF")["Predicted_Score"]
             .apply(lambda s: s.ewm(span=EMA_SMOOTH_SPAN, adjust=False).mean())
             .reset_index(level=0, drop=True)
    )

# REL SHAP for suffixes (demean within date); only for selected suffixes
suffix_shap_cols = [f"SHAP_{s}" for s in SELECTED_SUFFIXES if f"SHAP_{s}" in final.columns]
if suffix_shap_cols:
    rel = final.groupby("Date")[suffix_shap_cols].transform(lambda x: x.fillna(0.0) - x.fillna(0.0).mean())
    rel.columns = [c + "_REL" for c in rel.columns]
    final = pd.concat([final, rel], axis=1)

# Order: macro families (selected only) + suffix SHAPs + base/total + RELs
macro_shap_cols = [f"SHAP_{fam}" for fam in SELECTED_MACRO_COLS if f"SHAP_{fam}" in final.columns]
if macro_shap_cols:
    rel_macro = final.groupby("Date")[macro_shap_cols].transform(lambda x: x.fillna(0.0) - x.fillna(0.0).mean())
    rel_macro.columns = [c + "_REL" for c in rel_macro.columns]
    final = pd.concat([final, rel_macro], axis=1)

suffix_shap_cols = [f"SHAP_{s}" for s in SELECTED_SUFFIXES if f"SHAP_{s}" in final.columns]
base_total_cols = [c for c in ["SHAP_Base","SHAP_Total"] if c in final.columns]
rel_cols = [c for c in final.columns if c.startswith("SHAP_") and c.endswith("_REL")]

col_order = ["Date","ETF","Year","Actual_Return","Predicted_Return","Predicted_Score"] \
            + macro_shap_cols + suffix_shap_cols + base_total_cols + rel_cols

final = final[col_order].sort_values(["Date","ETF"])

# Final sanity: ensure no NaNs in Actual_Return
dropped = int(final["Actual_Return"].isna().sum())
if dropped > 0:
    print(f"[WARN] Found {dropped} NaNs in Actual_Return; dropping those rows.")
    final = final.dropna(subset=["Actual_Return"])

final.to_csv("stage1_predictions_with_shap_10ETFs.csv", index=False)
pd.DataFrame(metrics_rows).to_csv("stage1_unified_gate_metrics.csv", index=False)

print("Stage‑1 with unified SHAP gate complete. Wrote:")
print("  - stage1_predictions_with_shap_10ETFs.csv")
print("  - stage1_unified_gate_metrics.csv")



Unified gate selected suffixes: ['ATR', 'LagRet_1', 'LagRet_2', 'EMA_12', 'MACD', 'Mom_3']
Unified gate selected macro columns: ['Mkt-RF', 'CMA_lag_2', 'HML_lag_2', 'SMB_lag_2']

=== Stage‑1 (Unified‑gate) per‑ETF regression | window 2009 ===
Window 2009 done in 418.92s

=== Stage‑1 (Unified‑gate) per‑ETF regression | window 2010 ===
Window 2010 done in 460.69s

=== Stage‑1 (Unified‑gate) per‑ETF regression | window 2011 ===


C:\Users\Jiayang\AppData\Local\Temp\ipykernel_29064\929335556.py:112: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  ic, _ = spearmanr(y, p)
C:\Users\Jiayang\AppData\Local\Temp\ipykernel_29064\929335556.py:112: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  ic, _ = spearmanr(y, p)
C:\Users\Jiayang\AppData\Local\Temp\ipykernel_29064\929335556.py:112: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  ic, _ = spearmanr(y, p)
C:\Users\Jiayang\AppData\Local\Temp\ipykernel_29064\929335556.py:112: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  ic, _ = spearmanr(y, p)
C:\Users\Jiayang\AppData\Local\Temp\ipykernel_29064\929335556.py:112: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  ic, _ = spearmanr(y, p)
C:\Users\Jiayang\AppData\Local\Temp\ipykernel_2906

Window 2011 done in 424.74s

=== Stage‑1 (Unified‑gate) per‑ETF regression | window 2012 ===


C:\Users\Jiayang\AppData\Local\Temp\ipykernel_29064\929335556.py:112: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  ic, _ = spearmanr(y, p)
C:\Users\Jiayang\AppData\Local\Temp\ipykernel_29064\929335556.py:112: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  ic, _ = spearmanr(y, p)
C:\Users\Jiayang\AppData\Local\Temp\ipykernel_29064\929335556.py:112: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  ic, _ = spearmanr(y, p)
C:\Users\Jiayang\AppData\Local\Temp\ipykernel_29064\929335556.py:112: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  ic, _ = spearmanr(y, p)


Window 2012 done in 429.19s

=== Stage‑1 (Unified‑gate) per‑ETF regression | window 2013 ===


...

Window 2022 done in 386.14s

=== Stage‑1 (Unified‑gate) per‑ETF regression | window 2023 ===


C:\Users\Jiayang\AppData\Local\Temp\ipykernel_29064\929335556.py:112: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  ic, _ = spearmanr(y, p)
C:\Users\Jiayang\AppData\Local\Temp\ipykernel_29064\929335556.py:112: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  ic, _ = spearmanr(y, p)
C:\Users\Jiayang\AppData\Local\Temp\ipykernel_29064\929335556.py:112: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  ic, _ = spearmanr(y, p)
C:\Users\Jiayang\AppData\Local\Temp\ipykernel_29064\929335556.py:112: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  ic, _ = spearmanr(y, p)
C:\Users\Jiayang\AppData\Local\Temp\ipykernel_29064\929335556.py:112: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  ic, _ = spearmanr(y, p)
C:\Users\Jiayang\AppData\Local\Temp\ipykernel_2906

Window 2023 done in 369.17s

=== Stage‑1 (Unified‑gate) per‑ETF regression | window 2024 ===


C:\Users\Jiayang\AppData\Local\Temp\ipykernel_29064\929335556.py:112: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  ic, _ = spearmanr(y, p)
C:\Users\Jiayang\AppData\Local\Temp\ipykernel_29064\929335556.py:112: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  ic, _ = spearmanr(y, p)
C:\Users\Jiayang\AppData\Local\Temp\ipykernel_29064\929335556.py:112: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  ic, _ = spearmanr(y, p)
C:\Users\Jiayang\AppData\Local\Temp\ipykernel_29064\929335556.py:112: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  ic, _ = spearmanr(y, p)
C:\Users\Jiayang\AppData\Local\Temp\ipykernel_29064\929335556.py:112: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  ic, _ = spearmanr(y, p)
C:\Users\Jiayang\AppData\Local\Temp\ipykernel_2906

Window 2024 done in 412.62s
Stage‑1 with unified SHAP gate complete. Wrote:
  - stage1_predictions_with_shap_DIA_ETF.csv
  - stage1_unified_gate_metrics.csv


In [5]:
# ================================================
# Build Stage-2 daily observations (same structure)
# ================================================
import pandas as pd
import numpy as np

stage1_df = pd.read_csv("stage1_predictions_with_shap_10ETFs.csv", parse_dates=["Date"])
etfs = sorted(stage1_df["ETF"].unique())

# 1) Pivot to daily matrices
pred_pivot  = stage1_df.pivot(index="Date", columns="ETF", values="Predicted_Return").sort_index()
actual_pivot= stage1_df.pivot(index="Date", columns="ETF", values="Actual_Return").sort_index()

# 2) (Optional) standardize scores cross-sectionally per day for stability
pred_z = (pred_pivot - pred_pivot.mean(axis=1).values[:,None]) / (pred_pivot.std(axis=1).values[:,None] + 1e-8)

# 3) Volatility features from realized returns
vol_5 = actual_pivot.rolling(5).std()

# 4) Daily frame with required columns
dates = pred_pivot.index
agg = pd.DataFrame({"Date": dates})

for etf in etfs:
    agg[f"Predicted_Return_{etf}"] = pred_z[etf].values       # score (z-scored)
    agg[f"Actual_Return_{etf}"]    = actual_pivot[etf].values  # next-day realized
    agg[f"Volatility_{etf}"]       = vol_5[etf].values

# 5) SHAP aggregation — average & std across ETFs per feature name
#    We detect available SHAP_* columns automatically.
shap_cols = [c for c in stage1_df.columns if c.startswith("SHAP_")]
if shap_cols:
    # Melt to long then aggregate by Date and feature suffix
    long_shap = stage1_df[["Date"] + shap_cols].copy()
    long_shap = long_shap.melt(id_vars="Date", var_name="feat", value_name="shap")
    long_shap["suffix"] = long_shap["feat"].str.replace("^SHAP_", "", regex=True)
    shap_agg = (long_shap.groupby(["Date","suffix"])["shap"]
                        .agg(["mean","std"])
                        .reset_index())
    shap_mean = shap_agg.pivot(index="Date", columns="suffix", values="mean")
    shap_std  = shap_agg.pivot(index="Date", columns="suffix", values="std")
    shap_mean.columns = [f"Avg_SHAP_{c}" for c in shap_mean.columns]
    shap_std.columns  = [f"Std_SHAP_{c}" for c in shap_std.columns]
    shap_df = pd.concat([shap_mean, shap_std], axis=1).reindex(dates).ffill()
    agg = agg.merge(shap_df.reset_index(), on="Date", how="left")

# 6) Cross-sectional context features (as you had)
agg["CrossSec_Mean_PredRet"] = pred_z.mean(axis=1).values
agg["CrossSec_Std_PredRet"]  = pred_z.std(axis=1).values
agg["CrossSec_Mean_Volatility"] = vol_5.mean(axis=1).values

# 7) Per-ETF percentile ranks of the daily score
ranks = pred_z.rank(axis=1, pct=True)
for etf in etfs:
    agg[f"Rank_PredRet_{etf}"] = ranks[etf].values

# 8) Clean NA
vol_cols = [f"Volatility_{e}" for e in etfs]
agg[vol_cols] = agg[vol_cols].ffill()
agg.dropna(subset=vol_cols, inplace=True)

# 9) Save for Stage-2
agg.to_csv("stage2_rl_observations_optimized_10ETFs.csv", index=False)
print("Stage-2 observations written: stage2_rl_observations_optimized_10ETFs.csv")


Stage-2 observations written: stage2_rl_observations_optimized_DIA_ETF.csv


In [4]:
# # ================================================
# # STAGE 1 — Panel ranker (keeps Stage-2 interface)
# # ================================================
# import pandas as pd
# import numpy as np
# import xgboost as xgb
# import shap
# from sklearn.model_selection import ParameterGrid
# from scipy.stats import spearmanr
# import time
# import joblib
# import os, json, hashlib
# # -------- Controls --------
# FWD_HORIZON = 21          # ≈ one trading month forward
# USE_RISK_ADJ = True       # label = fwd_return / vol_20  (stabilizes ranks)
# EMA_SMOOTH_SPAN = 0       # smooth predictions per ETF (days); set 0 to disable
# SEEDS = [42, 202, 909]    # small bag for stability
# 
# # If you want *strict* determinism, prefer CPU hist. GPU can be slightly non-deterministic.
# USE_GPU = False  # set True only if you accept tiny non-determinism
# XGB_TREE_METHOD = "hist"
# XGB_DEVICE = "cuda" if USE_GPU else "cpu"
# 
# # Make a folder to keep all artifacts
# ART_DIR = "stage1_models"
# os.makedirs(ART_DIR, exist_ok=True)
#     
# RANK_PARAM_GRID = {
#     "max_depth": [4, 5],
#     "learning_rate": [0.05, 0.1],
#     "n_estimators": [600, 1000],
#     "subsample": [0.8, 0.9],
#     "colsample_bytree": [0.7, 0.9],
# }
# 
# TOP_N_FEATURES = 10      # total budget (informational)
# MIN_SUFFIXES   = 6       # ensure cross-sectional info
# MAX_GLOBALS    = 4       # macro families allowed
# 
# ETF_SUFFIXES, GLOBAL_FACTORS, shap_rank_table = run_shap_gate_select_topN(
#     features_df=features,
#     returns_df=returns,
#     etfs=etf_list,
#     start_year=start_year,
#     train_years=train_years,
#     valid_years=valid_years,
#     forward_h=FWD_HORIZON,
#     use_risk_adj=USE_RISK_ADJ,
#     top_n=TOP_N_FEATURES,
#     min_suffixes=MIN_SUFFIXES,
#     max_globals=MAX_GLOBALS,
#     seed=42
# )
# print("Gate chose ETF suffixes:", ETF_SUFFIXES)
# print("Gate chose GLOBAL_FACTORS:", GLOBAL_FACTORS)
# shap_rank_table.to_csv("gate_shap_ranking_full.csv")
# 
# # ETF-specific feature suffixes you already engineered
# 
# INCLUDE_VIX = False  # constant within day; not useful for within-day ranking
# pd.Series(ETF_SUFFIXES, name="ETF_SUFFIXES").to_csv("gate_selected_etf_suffixes.csv", index=False)
# pd.Series(GLOBAL_FACTORS, name="GLOBAL_FACTORS").to_csv("gate_selected_global_factors.csv", index=False)
# shap_rank_table.to_csv("gate_shap_ranking_full.csv")
# 
# def quick_hash(df: pd.DataFrame) -> str:
#     import hashlib
#     import numpy as np
# 
#     m = hashlib.md5()
# 
#     # index
#     idx = np.asarray(df.index.values)
#     m.update(idx.tobytes())
# 
#     # columns (as strings)
#     cols = np.asarray(df.columns.astype(str))
#     m.update(cols.tobytes())
# 
#     # values (force stable dtype)
#     vals = np.asarray(df.values, dtype=np.float64)
#     m.update(vals.tobytes())
# 
#     return m.hexdigest()[:10]
# 
# def save_window_artifacts(year: int,
#                           model: xgb.Booster,
#                           feat_cols: list,
#                           best_params: dict,
#                           ic_valid: float,
#                           date_ranges: dict,
#                           panel_hashes: dict) -> None:
#     win_dir = os.path.join(ART_DIR, f"year_{year}")
#     os.makedirs(win_dir, exist_ok=True)
# 
#     # Native XGB format (portable & stable)
#     model.save_model(os.path.join(win_dir, "xgb_ranker.json"))
# 
#     # Also save as binary booster if you ever want joblib
#     joblib.dump(model, os.path.join(win_dir, "xgb_ranker.booster"))
# 
#     meta = {
#         "best_params": best_params,
#         "ic_valid": float(ic_valid),
#         "feature_cols": list(feat_cols),
#         "date_ranges": date_ranges,
#         "panel_hashes": panel_hashes,
#         "xgb_tree_method": XGB_TREE_METHOD,
#         "xgb_device": XGB_DEVICE,
#         "seeded_bag": SEEDS,
#         "controls": {
#             "FWD_HORIZON": FWD_HORIZON,
#             "USE_RISK_ADJ": USE_RISK_ADJ,
#             "EMA_SMOOTH_SPAN": EMA_SMOOTH_SPAN
#         }
#     }
#     with open(os.path.join(win_dir, "meta.json"), "w") as f:
#         json.dump(meta, f, indent=2)
# 
# 
# def make_forward_returns(daily_ret_df: pd.DataFrame, horizon=21, method="compound"):
#     if method == "compound":
#         cum = (1.0 + daily_ret_df).cumprod()
#         return cum.shift(-horizon) / cum - 1.0
#     else:
#         return daily_ret_df.rolling(horizon).sum().shift(-horizon)
# 
# def build_panel(features_df: pd.DataFrame,
#                 returns_df: pd.DataFrame,
#                 etfs: list,
#                 start: pd.Timestamp,
#                 end: pd.Timestamp,
#                 forward_h=21,
#                 use_risk_adj=True,
#                 include_vix=False) -> pd.DataFrame:
#     """Rows = (Date, ETF).
#        Columns: selected ETF-specific generic columns + selected global factors.
#        Target y = forward_h return (optionally / vol_20).
#     """
#     feat = features_df.loc[start:end].copy()
#     rets = returns_df.loc[start:end].copy()
# 
#     fwd = _make_forward_returns(rets, horizon=forward_h, method="compound")
#     vol20 = rets.rolling(20).std()
# 
#     rows = []
#     for dt in feat.index:
#         if dt not in fwd.index:
#             continue
#         for etf in etfs:
#             row = {"Date": dt, "ETF": etf}
#             # ETF-specific (generic names, pull from per-ETF columns)
#             for suf in ETF_SUFFIXES:
#                 col = f"{etf}_{suf}"
#                 if col in feat.columns:
#                     row[suf] = feat.at[dt, col]
#             # Global macro
#             for gf in GLOBAL_FACTORS:
#                 if gf in feat.columns:
#                     row[gf] = feat.at[dt, gf]
#             if include_vix and "VIX" in feat.columns:
#                 row["VIX"] = feat.at[dt, "VIX"]
# 
#             yy = fwd.at[dt, etf] if pd.notna(fwd.at[dt, etf]) else np.nan
#             if use_risk_adj:
#                 vv = vol20.at[dt, etf]
#                 yy = np.nan if (pd.isna(yy) or pd.isna(vv)) else (yy / max(vv, 1e-6))
#             row["y"] = yy
#             # inside the per-ETF loop in build_panel, before appending row:
#             missing = [f"{etf}_{s}" for s in ETF_SUFFIXES if f"{etf}_{s}" not in feat.columns]
#             if missing:
#                 raise RuntimeError(f"Missing ETF-specific columns for {etf}: {missing[:3]} ...")
#             rows.append(row)
# 
#     panel = pd.DataFrame(rows).dropna()
#     if panel.empty:
#         raise ValueError("Panel dataset is empty; check date ranges or gate selection.")
# 
#     feat_cols = [c for c in panel.columns if c not in ["Date","ETF","y"]]
#     # Only z-score ETF-specific cols (not global):
#     cs_cols = [c for c in feat_cols if c not in GLOBAL_FACTORS and c != "VIX"]
#     panel[cs_cols] = (
#         panel.groupby("Date")[cs_cols]
#              .transform(lambda x: (x - x.mean()) / (x.std(ddof=0) + 1e-4))
#     )
# 
#     # Mild winsorization of ETF-specific cols
#     for c in cs_cols:
#         q1, q99 = panel[c].quantile([0.01, 0.99])
#         panel[c] = panel[c].clip(q1, q99)
# 
#     panel.sort_values(["Date","ETF"], inplace=True)
#     panel.reset_index(drop=True, inplace=True)
#     return panel
# 
# 
# def groups_from_dates(date_series: pd.Series) -> np.ndarray:
#     # XGB ranker groups = items per date
#     return date_series.value_counts().sort_index().values.astype(int)
# 
# # --------- Rolling training as REGRESSOR; write daily outputs + SHAP ----------
# 
# from scipy.stats import spearmanr
# 
# # Keep your mean_daily_ic helper
# def mean_daily_ic(dates: np.ndarray, y_true: np.ndarray, y_pred: np.ndarray) -> float:
#     df = pd.DataFrame({"Date": dates, "y": y_true, "p": y_pred})
#     vals = []
#     for dt, g in df.groupby("Date"):
#         yv = g["y"].values
#         pv = g["p"].values
#         m = np.isfinite(yv) & np.isfinite(pv)
#         yv, pv = yv[m], pv[m]
#         if len(yv) >= 3:
#             # skip days where either side is constant
#             if np.std(yv) < 1e-12 or np.std(pv) < 1e-12:
#                 continue
#             ic, _ = spearmanr(yv, pv)
#             if np.isfinite(ic): vals.append(ic)
#     return float(np.mean(vals)) if vals else np.nan
# 
# 
# # Small param grid for regression (you can tune more later)
# REG_PARAM_GRID = {
#     "max_depth": [4, 5],
#     "learning_rate": [0.05, 0.1],
#     "n_estimators": [600, 1000],
#     "subsample": [0.8, 0.9],
#     "colsample_bytree": [0.7, 0.9],
#     "min_child_weight": [1.0],
#     "reg_lambda": [1.0],
# }
# 
# # Map global macro columns into factor families for SHAP aggregation
# FACTOR_FAMILY_MAP = {
#     "MKT": ["Mkt-RF", "Mkt-RF_lag_1", "Mkt-RF_lag_2", "Mkt-RF_lag_3"],
#     "SMB": ["SMB", "SMB_lag_1", "SMB_lag_2", "SMB_lag_3"],
#     "HML": ["HML", "HML_lag_1", "HML_lag_2", "HML_lag_3"],
#     "RMW": ["RMW", "RMW_lag_1", "RMW_lag_2", "RMW_lag_3"],
#     "CMA": ["CMA", "CMA_lag_1", "CMA_lag_2", "CMA_lag_3"],
#     # If you included VIX in features, this will be aggregated alone
#     "VIX": ["VIX"]
# }
# 
# def aggregate_shap_df(shap_df: pd.DataFrame) -> pd.DataFrame:
#     """Return a DataFrame with SHAP_MKT/SMB/... + SHAP_<ETF suffix> columns."""
#     out = pd.DataFrame(index=shap_df.index)
# 
#     # Macro families (sum across available lag columns)
#     for fam, cols in FACTOR_FAMILY_MAP.items():
#         use = [c for c in cols if c in shap_df.columns]
#         if len(use) > 0:
#             out[f"SHAP_{fam}"] = shap_df[use].sum(axis=1)
# 
#     # ETF‑specific engineered suffixes (keep them one by one)
#     for suf in ETF_SUFFIXES:
#         if suf in shap_df.columns:
#             out[f"SHAP_{suf}"] = shap_df[suf]
# 
#     # Total additivity check helper (optional but nice to have)
#     out["SHAP_Total"] = shap_df.sum(axis=1)
#     return out
# 
# # ------------- rolling windows -------------
# all_predictions = []
# metrics_log = []
# 
# year = start_year
# while year <= end_year - test_years + 1:
#     print(f"\n=== Stage‑1 panel REGRESSOR | window starting {year} ===")
#     t0 = time.time()
# 
#     train_start = pd.Timestamp(year - train_years, 1, 1)
#     train_end   = pd.Timestamp(year - valid_years - 1, 12, 31)
#     valid_start = pd.Timestamp(year - valid_years, 1, 1)
#     valid_end   = pd.Timestamp(year - 1, 12, 31)
#     test_start  = pd.Timestamp(year, 1, 1)
#     test_end    = pd.Timestamp(year + test_years - 1, 12, 31)
# 
#     # Build panels (same as before)
#     panel_tr = build_panel(features, returns, etf_list, train_start, train_end,
#                            forward_h=FWD_HORIZON, use_risk_adj=USE_RISK_ADJ, include_vix=INCLUDE_VIX)
#     panel_va = build_panel(features, returns, etf_list, valid_start, valid_end,
#                            forward_h=FWD_HORIZON, use_risk_adj=USE_RISK_ADJ, include_vix=INCLUDE_VIX)
#     panel_te = build_panel(features, returns, etf_list, test_start,  test_end,
#                            forward_h=FWD_HORIZON, use_risk_adj=USE_RISK_ADJ, include_vix=INCLUDE_VIX)
# 
#     feat_cols = [c for c in panel_tr.columns if c not in ["Date","ETF","y"]]
# 
#     X_tr, y_tr = panel_tr[feat_cols].values, panel_tr["y"].values
#     X_va, y_va = panel_va[feat_cols].values, panel_va["y"].values
#     X_te, y_te = panel_te[feat_cols].values, panel_te["y"].values
# 
#     # ----- model selection: maximize validation IC, tie‑break with RMSE -----
#     best_ic, best_rmse = -np.inf, np.inf
#     best_model, best_params = None, None
# 
#     for params in ParameterGrid(REG_PARAM_GRID):
#         for seed in SEEDS:
#             dtr, dva = xgb.DMatrix(X_tr, label=y_tr), xgb.DMatrix(X_va, label=y_va)
# 
#             num_boost_round = int(params["n_estimators"])
#             xgb_params = {
#                 "objective": "reg:squarederror",
#                 "tree_method": XGB_TREE_METHOD,
#                 "eval_metric": "rmse",
#                 "seed": int(seed),
#                 "sampling_method": "uniform",
#                 "max_depth": int(params["max_depth"]),
#                 "eta": float(params["learning_rate"]),
#                 "subsample": float(params["subsample"]),
#                 "colsample_bytree": float(params["colsample_bytree"]),
#                 "min_child_weight": float(params["min_child_weight"]),
#                 "lambda": float(params.get("reg_lambda", 1.0)),
#                 "base_score": float(np.nanmean(y_tr)),
#             }
# 
#             model = xgb.train(
#                 xgb_params,
#                 dtr,
#                 num_boost_round=max(200, num_boost_round),
#                 evals=[(dtr, "train"), (dva, "valid")],
#                 early_stopping_rounds=min(100, num_boost_round // 3),
#                 verbose_eval=False
#             )
# 
#             # Validation prediction
#             p_va = model.predict(dva, iteration_range=(0, (model.best_iteration or 0) + 1))
#             ic_va = mean_daily_ic(panel_va["Date"].values, y_va, p_va)
#             rmse_va = float(np.sqrt(np.mean((y_va - p_va) ** 2)))
# 
#             better = (ic_va > best_ic) or (np.isclose(ic_va, best_ic) and rmse_va < best_rmse)
#             if better:
#                 best_ic, best_rmse = ic_va, rmse_va
#                 best_model = model
#                 best_params = {
#                     "seed": int(seed),
#                     "n_estimators": int(num_boost_round),
#                     "max_depth": int(params["max_depth"]),
#                     "learning_rate": float(params["learning_rate"]),
#                     "subsample": float(params["subsample"]),
#                     "colsample_bytree": float(params["colsample_bytree"]),
#                     "min_child_weight": float(params["min_child_weight"]),
#                     "reg_lambda": float(params.get("reg_lambda", 1.0)),
#                 }
# 
#     print(f"Best validation IC = {best_ic:.3f} | RMSE = {best_rmse:.5f} with {best_params}")
# 
#     # --- Predict on TEST (raw regression + daily ranks) ---
#     dte = xgb.DMatrix(X_te)
#     p_te = best_model.predict(dte, iteration_range=(0, (best_model.best_iteration or 0) + 1))
# 
#     # Raw predicted forward (risk‑adjusted) return
#     test_preds = panel_te[["Date","ETF"]].copy()
#     test_preds["Predicted_FwdRet"] = p_te
# 
#     # Cross‑sectional score per date (ranked to [-0.5, 0.5])
#     test_preds["Predicted_Score"] = (
#         test_preds.groupby("Date")["Predicted_FwdRet"]
#                   .transform(lambda s: s.rank(pct=True) - 0.5)
#     )
# 
#     # Optional smoothing within ETF (no leakage within test)
#     if EMA_SMOOTH_SPAN and EMA_SMOOTH_SPAN > 1:
#         test_preds["Predicted_Score"] = (
#             test_preds.groupby("ETF")["Predicted_Score"]
#                       .apply(lambda s: s.ewm(span=EMA_SMOOTH_SPAN, adjust=False).mean())
#                       .reset_index(level=0, drop=True)
#         )
# 
#     # --- Compute SHAP on TEST set ---
#     # IMPORTANT: SHAP works now because we trained a regression model.
#     explainer = shap.TreeExplainer(best_model)
#     shap_values_te = explainer.shap_values(X_te)              # (n_test, n_features)
#     expected_value = explainer.expected_value                  # scalar for regression
#     if np.ndim(expected_value) == 0:
#         expected_value = float(expected_value)
#         base_vec = np.full(len(panel_te), expected_value, dtype=float)
#     else:
#         base_vec = np.full(len(panel_te), float(expected_value[0]), dtype=float)
# 
#     shap_df_te = pd.DataFrame(shap_values_te, columns=feat_cols, index=panel_te.index)
#     shap_agg_te = aggregate_shap_df(shap_df_te)
#     shap_agg_te["SHAP_Base"] = base_vec
# 
#     suf_cols = [f"SHAP_{s}" for s in ETF_SUFFIXES if f"SHAP_{s}" in shap_agg_te.columns]
#     if suf_cols:
#         shap_rel = shap_agg_te[suf_cols].copy()
#         shap_rel = shap_rel.groupby(panel_te["Date"]).transform(lambda x: x - x.mean())
#         shap_rel.columns = [c + "_REL" for c in shap_rel.columns]
#         shap_agg_te = pd.concat([shap_agg_te, shap_rel], axis=1)
#     
#     # --- Prepare Stage‑2 schema (+ keep raw regression pred for analysis) ---
#     test_out = test_preds[["Date","ETF"]].copy()
#     # Keep Stage‑2 compatibility: Predicted_Return = cross‑sectional score;
#     # also export the raw regression prediction as Predicted_FwdRet for your diagnostics.
#     test_out["Predicted_Return"] = test_preds["Predicted_Score"].values
#     test_out["Predicted_FwdRet"] = test_preds["Predicted_FwdRet"].values
# 
#     # Actual next‑day (unchanged)
#     next_day_actual = returns.shift(-1)
#     test_out["Actual_Return"] = [
#         next_day_actual.at[dt, etf] for dt, etf in zip(test_out["Date"], test_out["ETF"])
#     ]
#     test_out["Year"] = year
# 
#     # Attach SHAP aggregates
#     test_out = pd.concat([test_out, shap_agg_te.reset_index(drop=True)], axis=1)
# 
#     all_predictions.append(test_out)
# 
#     # Log window test metrics
#     ic_te = mean_daily_ic(panel_te["Date"].values, y_te, p_te)
#     rmse_te = float(np.sqrt(np.mean((y_te - p_te) ** 2)))
#     metrics_log.append({
#         "Year": year, "IC_valid": float(best_ic), "RMSE_valid": float(best_rmse),
#         "IC_test": float(ic_te), "RMSE_test": float(rmse_te),
#         "train_range": f"{train_start.date()}..{train_end.date()}",
#         "valid_range": f"{valid_start.date()}..{valid_end.date()}",
#         "test_range":  f"{test_start.date()}..{test_end.date()}"
#     })
# 
#     # Save artifacts (unchanged helper)
#     save_window_artifacts(
#         year=year,
#         model=best_model,
#         feat_cols=feat_cols,
#         best_params=best_params,
#         ic_valid=best_ic,
#         date_ranges={
#             "train": [str(train_start.date()), str(train_end.date())],
#             "valid": [str(valid_start.date()), str(valid_end.date())],
#             "test":  [str(test_start.date()),  str(test_end.date())],
#         },
#         panel_hashes={
#             "train": quick_hash(panel_tr[feat_cols]),
#             "valid": quick_hash(panel_va[feat_cols]),
#             "test":  quick_hash(panel_te[feat_cols]),
#         }
#     )
# 
#     print(f"Window processed in {(time.time()-t0):.2f}s | Test IC={ic_te:.3f} RMSE={rmse_te:.5f}")
#     year += retrain_frequency
# 
# # ---- Final save (now includes SHAP_* and the raw regression prediction) ----
# final_predictions_df = pd.concat(all_predictions, ignore_index=True)
# 
# # Put SHAP columns at the end
# shap_cols = [c for c in final_predictions_df.columns if c.startswith("SHAP_")]
# col_order = ["Date","ETF","Year","Actual_Return","Predicted_Return","Predicted_FwdRet"] + shap_cols
# final_predictions_df = final_predictions_df[col_order]
# final_predictions_df.to_csv("stage1_predictions_with_shap_10ETFs.csv", index=False)
# 
# pd.DataFrame(metrics_log).to_csv("stage1_regression_window_metrics.csv", index=False)
# print("Stage 1 (panel regressor) completed and data saved for Stage 2.")
